In [73]:
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch import nn
from torch.nn import init, MarginRankingLoss
from transformers import BertModel, RobertaModel
from transformers import BertTokenizer, RobertaTokenizer
from torch.optim import Adam
from distutils.version import LooseVersion
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from torch.autograd import Variable
from transformers import AutoConfig, AutoModel, AutoTokenizer
import nltk
import re
import Levenshtein
import spacy
import en_core_web_sm
import torch.optim as optim
from torch.distributions import Categorical
from numpy import linalg as LA
from transformers import AutoModelForMaskedLM
from nltk.corpus import wordnet
import torch.nn.functional as F
import random
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import precision_recall_fscore_support
from nltk.corpus import words as wal
from sklearn.utils import resample


In [56]:
class MyDataset(Dataset):
    def __init__(self,file_name):
        df1 = pd.read_csv(file_name)
        df1 = df1.fillna("")
        res = df1['X']
#         ab = df1['X']
#         res = [sub.replace("<mask>", "[MASK]") for sub in ab]
        self.X_list = res
        self.y_list = df1['y']
    def __len__(self):
        return len(self.X_list)
    def __getitem__(self,idx):
        mapi = []
        mapi.append(self.X_list[idx])
        mapi.append(self.y_list[idx])
        return mapi

In [59]:
class Step1_model(nn.Module):
    def __init__(self, hidden_size=512):
        super(Step1_model, self).__init__()
        self.hidden_size = hidden_size
#         self.model = AutoModel.from_pretrained("roberta-base")
#         self.tokenizer = AutoTokenizer.from_pretrained("roberta-base")
#         self.config = AutoConfig.from_pretrained("roberta-base")
        self.model = RobertaForSequenceClassification.from_pretrained("microsoft/graphcodebert-base", num_labels=6)
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
        self.config = AutoConfig.from_pretrained("microsoft/graphcodebert-base")

#         self.model = AutoModelForMaskedLM.from_pretrained('bert-base-cased')
#         self.tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#         self.config = AutoConfig.from_pretrained("bert-base-cased")
        for name, param in self.model.named_parameters():
            param.requires_grad = True
            if name.startswith("model.roberta.encoder.layer.0"):
                param.requires_grad = False
            if name.startswith("model.roberta.encoder.layer.1"):
                param.requires_grad = False
            if name.startswith("model.roberta.encoder.layer.2"):
                param.requires_grad = False
            if name.startswith("model.roberta.encoder.layer.3"):
                param.requires_grad = False
            if name.startswith("model.roberta.encoder.layer.4"):
                param.requires_grad = False
            if name.startswith("model.roberta.encoder.layer.5"):
                param.requires_grad = False
            if name.startswith("model.roberta.encoder.layer.6"):
                param.requires_grad = False
            if name.startswith("model.roberta.encoder.layer.7"):
                param.requires_grad = False

    def forward(self, mapi):
        X_init = mapi[0]
        y = mapi[1]
        print(y)
        nl = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', y)
        lb = ' '.join(nl).lower()
        x = tokenizer.tokenize(lb)
        nlab = len(x)
        print(nlab)
        tokens = self.tokenizer.encode_plus(X_init, add_special_tokens=False,return_tensors='pt')
        input_id_chunki = tokens['input_ids'][0].split(510)
        input_id_chunks = []
        mask_chunks  = []
        mask_chunki = tokens['attention_mask'][0].split(510)
        for tensor in input_id_chunki:
            input_id_chunks.append(tensor)
        for tensor in mask_chunki:
            mask_chunks.append(tensor)
        xi = torch.full((1,), fill_value=101)
        yi = torch.full((1,), fill_value=1)
        zi = torch.full((1,), fill_value=102)
        for r in range(len(input_id_chunks)):
            input_id_chunks[r] = torch.cat([xi, input_id_chunks[r]],dim = -1)
            input_id_chunks[r] = torch.cat([input_id_chunks[r],zi],dim=-1)
            mask_chunks[r] = torch.cat([yi, mask_chunks[r]],dim=-1)
            mask_chunks[r] = torch.cat([mask_chunks[r],yi],dim=-1)
        di = torch.full((1,), fill_value=0)
        for i in range(len(input_id_chunks)):
            # get required padding length
            pad_len = 512 - input_id_chunks[i].shape[0]
            # check if tensor length satisfies required chunk size
            if pad_len > 0:
                # if padding length is more than 0, we must add padding
                for p in range(pad_len):
                    input_id_chunks[i] = torch.cat([input_id_chunks[i],di],dim=-1)
                    mask_chunks[i] = torch.cat([mask_chunks[i],di],dim=-1)
        input_ids = torch.stack(input_id_chunks)
        attention_mask = torch.stack(mask_chunks)
        input_dict = {
            'input_ids': input_ids.long(),
            'attention_mask': attention_mask.int()
        }
        with torch.no_grad():
            outputs = self.model(**input_dict)
        last_hidden_state = outputs.logits.squeeze()
        lhs_agg = []
        if len(last_hidden_state) == 1:
            lhs_agg.append(last_hidden_state)
        else:
            for p in range(len(last_hidden_state)):
                lhs_agg.append(last_hidden_state[p])
        lhs = lhs_agg[0]
        for i in range(len(lhs_agg)):
            if i == 0:
                continue
            lhs+=lhs_agg[i]
        lhs/=len(lhs_agg)
        print(lhs)
        predicted_prob = torch.softmax(lhs, dim=0)
        pred = torch.argmax(predicted_prob).item()
        pred+=1
        print(pred)
        criterion = torch.nn.CrossEntropyLoss()
        predicted = torch.tensor([pred], dtype = float)  # Wrap the predicted label in a tensor
        actual = torch.tensor([nlab], dtype = float)  # Wrap the actual label in a tensor
        l1 = Variable(torch.tensor([(actual-predicted)**2],dtype=float),requires_grad = True)
        return {'loss':l1, 'actual_pred':pred}

In [60]:
epoch_number = 0
EPOCHS = 5
run_int = 0
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
model = Step1_model()
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
myDs=MyDataset('dat.csv')
# vDs=MyDataset('valid_sing_cand.csv')
train_loader=DataLoader(myDs,batch_size=64,shuffle=True)
best_loss = torch.full((1,), fill_value=100000)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [61]:
flag = 0
def train_one_epoch(transformer_model, dataset):
    global flag
    for batch in dataset:
        p = 0
        inputs = batch
        optimizer.zero_grad()
#         try:
        for i in range(len(inputs[0])):
            l = []
            l.append(inputs[0][i])
            l.append(inputs[1][i])
            opi = transformer_model(l)
            loss1 = opi['loss']
            if p == 0:
                loss = Variable(loss1, requires_grad = True)
                p+=1
            else:
                loss = torch.cat([loss, loss1],dim = -1)
                p+=1
#         except:
#             continue
        loss.sum().backward()
        optimizer.step()
        if p % 1 == 0:
            print('  batch loss: {}'.format(loss))
    return loss.sum()

In [62]:
for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(model,train_loader)

    # We don't need gradients on to do reporting
    model.train(False)

#     running_vloss = 0.0
#     for i, vdata in enumerate(validation_loader):
#         try:
#             vinputs, vlabels = vdata
#             voutputs = model(vinputs)
#             vloss = loss_fn(voutputs, vlabels)
#             running_vloss += vloss
#         except:
#             flag+=1
#             continue

#     avg_vloss = running_vloss / (i + 1)
#     print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
    print('LOSS train {}'.format(avg_loss))
    # Log the running loss averaged per batch
    # for both training and validation
#     writer.add_scalars('Training vs. Validation Loss',
#                     { 'Training' : avg_loss, 'Validation' : avg_vloss },
#                     epoch_number + 1)
#     writer.add_scalars('Training Loss',
#                     { 'Training' : avg_loss},
#                     epoch_number + 1)
#     writer.flush()

    # Track best performance, and save the model's state
    if avg_loss < best_loss:
        best_loss = avg_loss
        model_path = 'var_runs_class/model_{}_{}'.format(run_int, epoch_number)
        torch.save(model.state_dict(), model_path)
#     if avg_vloss < best_vloss:
#         best_vloss = avg_vloss
#         model_path = 'model_{}_{}'.format(timestamp, epoch_number)
#         torch.save(model.state_dict(), model_path)


    epoch_number += 1

Token indices sequence length is longer than the specified maximum sequence length for this model (1097 > 512). Running this sequence through the model will result in indexing errors


EPOCH 1:
subscriber

3
tensor([ 0.1507, -0.0968,  0.0036,  0.0029, -0.0912,  0.0173, -0.1393, -0.0378])
1
expected

1
tensor([ 0.0411, -0.0696, -0.0522, -0.0773, -0.0898, -0.0337, -0.0662, -0.0220])
1
expectedValues

2
tensor([ 0.0201,  0.0071, -0.0399, -0.0169, -0.0621, -0.0241, -0.0215, -0.0814])
1
bytes

1
tensor([ 0.0864, -0.1086,  0.0376, -0.0619, -0.1122, -0.0584, -0.0012, -0.0468])
1
bitmap

2
tensor([ 0.0857, -0.0955,  0.0146, -0.0521, -0.0925, -0.0928, -0.0261, -0.0113])
1
expected

1
tensor([-0.0216, -0.0823, -0.1127, -0.0611, -0.0750, -0.0005, -0.1237, -0.0744])
6
delayTime

2
tensor([ 0.0788, -0.0013, -0.1026, -0.0105, -0.0525, -0.0205, -0.1085, -0.0867])
1
second

1
tensor([ 0.1067,  0.0248, -0.0211, -0.0237, -0.1405, -0.0275, -0.0203, -0.0439])
1
filled

1
tensor([ 0.0940, -0.0032, -0.1279, -0.0944, -0.0859,  0.0532, -0.0733, -0.1083])
1
instance

1
tensor([ 0.0858,  0.0202,  0.0191,  0.0282, -0.0979, -0.0748, -0.0297, -0.0675])
1
missed

2
tensor([ 0.0532, -0.0560, -0.06

KeyboardInterrupt: 